# Hyperparameter Optimization with Emukit

## Overview

In [1]:
# General imports and parameters of figures should be loaded at the beginning of the overview

### General imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

### --- Figure config
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
LEGEND_SIZE = 15
TITLE_SIZE = 25
AXIS_SIZE = 15

In this tutorial we will learn how we can use Bayesian optimization to tune the hyperparameters of a small feed forward neural network on MNIST. We will also show how we can speed up the optimization process by reasoning across training dataset subsets.

Before you start, make sure that you installed the following dependencies:
- numpy (pip install numpy)
- matplotlib (pip install matplotlib)
- keras (pip install keras)

### Navigation

1. [Section 1](#1.-Section-1)

2. [Section 2](#2.-Section-2)

3. [Conclusions](#3.-Conclusions)

4. [References](#4.-References)

# 1. Continuous Hyperparameter Optimization

In this tutorial we learn how we can use Emukit to optimize the learning rate, batch size and the dropout rates of a 2-layer feed-forward neural network on MNIST. For simplicity we bootstrap from the MNIST [example](https://github.com/keras-team/keras/blob/master/examples/mnist_mlp.py) of Keras.  
The code implements the objective function, which get as input a hyperparameter configuration, trains the network with these hyperparameters and finally returns the validation error.

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop


def train_and_validate(hyperparameters):
    print(hyperparameters)
    return np.random.randn(1,1)
    batch_size = 128
    num_classes = 10
    epochs = 20

    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    
    return 1 - score

To allow for an easy integration into the emukit, we wrap the objective function with the UserFunction interface.

In [2]:
from emukit.core.loop.user_function import UserFunctionWrapper

objective_function = UserFunctionWrapper(train_and_validate)

Using TensorFlow backend.


Besides the objective funtion, we need to define the configuration space, or input space which defines the ranges for all hyperparameters. 
Note that, it is usually good practice to optimize the learning rate on a logarithmic scale rather than a linear scale, thus, we optimize the exponent $\gamma \in [-6, -1]$ and set the learning rate in the objective function $10^{\gamma}$.

In [3]:
from emukit.core import ContinuousParameter, ParameterSpace

list_params = []
list_params.append(ContinuousParameter("learning_rate",-6, -1))
list_params.append(ContinuousParameter("batch_size",32, 512))
list_params.append(ContinuousParameter("dropout_1", 0, .99))
list_params.append(ContinuousParameter("dropout_2", 0, .99))

configuration_space = ParameterSpace(list_params)

# 2. Random Search

To get started, we will first look at random search [[Bergstra et al, 2012]](#4.-References), which is, besides its simplicity, usually a quite tough baseline to beat. In general, if you develop a new hyperparameter optimization method, you should always first compare against random search to see whether your method actually works or if there is still a bug somewhere.

The following function draws `n_iters` hyperparameter configurations uniformly at random from the configuration space

In [4]:
from emukit.benchmarking.loop_benchmarking.random_search import RandomSearch
rs = RandomSearch(configuration_space)
rs.run_loop(objective_function, 5)

[[-5.57514487e+00  4.57510122e+02  8.57193240e-01  3.65013894e-01]]
[[ -1.93383353 261.44480403   0.52089088   0.68185324]]
[[ -1.65343417 279.5912788    0.63661122   0.71218883]]
[[-2.83757350e+00  4.40945941e+02  1.10851391e-01  9.71881406e-01]]
[[-4.22356587e+00  4.65442396e+02  3.86356272e-01  1.04874499e-01]]


# 3. Bayesian optimization

## Initial Design

After we familiarized ourself with the problem, we will now look at how we can use Emukit's Bayesian optimization module to optimize the hyperparameters.
For more details about Bayesian optimization and how it is implemented in Emukit have a look at this [tutorial](https://github.com/amzn/emukit/blob/master/notebooks/Emukit-tutorial-Bayesian-optimization-introduction.ipynb) here.


Bayesian optimization uses a model to guide the search. However, to train a model we first need data, which we do not have in the beginning. Thus, we will first use a so called initial design to collect some data point before we start the Bayesian optimization loop. Here we will simply draw and evaluate N random configuration. However, one could do more sophisticated things such as es experimental design or multi-task learning to warmstart the optimization procedure.

In [2]:
from emukit.experimental_design.model_free.random_design import RandomDesign
n_init = 2

initial_design = RandomDesign(configuration_space)

X_init = initial_design.get_samples(n_init)
Y_init = np.zeros([n_init, 1])
for i in range(n_init):
    Y_init[i] = objective_function.evaluate(X_init[i, None])[0].Y


NameError: name 'configuration_space' is not defined

Now we define all the ingredients to run the main loop. Since all hyperparameters are continuous we use a Gaussian process (GP) as probabilistic model of the objective function and expected improvement as acquisition function. Following [[Snoek et al, 2012]](#4.-References) we integrate the acquisition function over the GP hyperparameters.

In [7]:
import GPy

from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.acquisition import IntegratedHyperParameterAcquisition
from emukit.model_wrappers import GPyModelWrapper

kernel = GPy.kern.Matern52(X_init.shape[1], variance=1., ARD=False)
gpmodel = GPy.models.GPRegression(X_init, Y_init, kernel)

model = GPyModelWrapper(gpmodel)

acquisition_generator = lambda model: ExpectedImprovement(model)
acquisition = IntegratedHyperParameterAcquisition(model, acquisition_generator)

bo =  BayesianOptimizationLoop(space=configuration_space, model=model, acquisition=acquisition)

In [ ]:
Now we are all set and can 

In [8]:
bo.run_loop(objective_function, 5)

Optimization restart 1/1, f = 4.57633697608823
[[-4.47031553e+00  5.11556372e+02  4.34035070e-01  2.76702315e-01]]
Optimization restart 1/1, f = 6.257839491834606
[[-4.29247068 32.42386657  0.2527486   0.5821326 ]]
Optimization restart 1/1, f = 7.9521390370070915
[[ -5.34278582 233.57920905   0.29280434   0.97328834]]
Optimization restart 1/1, f = 9.520802955249184
[[-1.35853062e+00  4.52192784e+02  9.07874679e-02  6.57912882e-01]]
Optimization restart 1/1, f = 11.22066770216205
[[-1.35288357e+00  4.52197706e+02  8.95969815e-02  6.57448270e-01]]
Optimization restart 1/1, f = 12.619679708480689


# 3. Fabolas: Fast Bayesian Optimization on Large Datasets

In [13]:
import time

def train_and_validate_dataset_size(hyperparameters, dataset_size):
    print(hyperparameters, dataset_size)
    return np.random.randn(1,1)

    start_time = time.time()
    batch_size = 128
    num_classes = 10
    epochs = 20

    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    
    training_time = time.time() - start_time
    
    return 1 - score, training_time
    
    
    return np.array([[y]]), np.array([[c]])

s_min = 100
s_max = 50000

In [10]:

# initial_design = LatinDesign(space)

# grid = initial_design.get_samples(n_init)
# X_init = np.zeros([n_init, grid.shape[1] + 1])
# Y_init = np.zeros([n_init, 1])
# cost_init = np.zeros([n_init])

# subsets = np.array([s_max // 2 ** i for i in range(2, 10)])[::-1]
# idx = np.where(subsets < s_min)[0]

# subsets[idx] = s_min

# for it in range(n_init):
#     func_val, cost = func(x=grid[it], s=subsets[it % len(subsets)])

#     X_init[it] = np.concatenate((grid[it], np.array([subsets[it % len(subsets)]])))
#     Y_init[it] = func_val
#     cost_init[it] = cost


# loop = FabolasLoop(X_init=X_init, Y_init=Y_init, cost_init=cost_init, space=space, s_min=s_min,
#                    s_max=s_max, marginalize_hypers=marginalize_hypers)
loop = FabolasLoop(X_init=None, Y_init=None, cost_init=None, space=ParameterSpacespace, s_min=s_min,
                   s_max=s_max, marginalize_hypers=marginalize_hypers)

loop.run_loop(user_function=UserFunctionWrapper(train_and_validate_dataset_size),
              stopping_condition=FixedIterationsStoppingCondition(n_iters - n_init))

NameError: name 'LatinDesign' is not defined

# 4. Discrete Hyperparameter Optimization and Architecture Search

# 5. References

- Kennedy, M.C. and O'Hagan, A., 2000. *Predicting the output from a complex computer code when fast approximations are available.* Biometrika, 87(1), pp.1-13.